# Prepare Single Note Brief Hospital Course Dataset

This notebook prepares the preceding text in the discharge summary as training data for the brief hospital course, including no other data as a baseline. 

In [1]:
import pandas as pd
import re
#import tiktoken

In [2]:
#pip install tiktoken

In [6]:
challenge_data_fp = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/"
#project/test.zip

In [10]:
#target_train = pd.read_csv(challenge_data_fp + "train/discharge_target.csv.gz", keep_default_na=False)
#target_valid = pd.read_csv(challenge_data_fp + "valid/discharge_target.csv.gz", keep_default_na=False)
target_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge_target.csv.gz", keep_default_na=False)


In [11]:
target_test

,note_id,hadm_id,discharge_instructions,brief_hospital_course,discharge_instructions_word_count,brief_hospital_course_word_count
0,19766998-DS-20,26231944,"Dear Mr. ___,\n\nIt was a pleasure to care for...","___ with PMH HCV, ETOH cirrhosis with ascites,...",145,62
1,10336082-DS-11,28542384,"Dear Ms. ___, \n We had the pleasure of taking...",___ yo F with recent lumbar laminectomy c/b MS...,76,466
2,10481170-DS-22,26489329,You were admitted to the hospital after a with...,The patient was seen ___ the emergency departm...,396,262
3,11576109-DS-14,22641254,"Dear Ms. ___, \n\nIt was a pleasure caring for...","Ms. ___ is a ___ woman with a history of T2DM,...",142,496
4,19249697-DS-17,29265750,"Dear Mr. ___, \n\nYou were admitted to ___ wit...",___ with IDDM and h/o provoked PE on apixiban ...,132,396
...,...,...,...,...,...,...
14697,18995174-DS-17,27445071,"Dear Mr. ___,\n\nYou were admitted after getti...",___ M PMHx dilated non-ischemic cardiomyopathy...,128,216
14698,13588195-DS-9,26192891,"Dear Mr. ___,\n\n You were admitted to ___ af...","Mr. ___ is a ___ year-old man with CAD, HTN, H...",47,40
14699,10873131-DS-17,26584893,"Dear Ms. ___,\n\nIt was a pleasure taking ___ ...",___ year old female with past medical history ...,260,1014
14700,12332377-DS-11,25623241,"Dear Mr. ___, \n\nYou were admitted to ___ \nb...",Mr. ___ is a ___ male with a past medical hist...,244,462


In [14]:
#target_train = pd.read_csv(challenge_data_fp + "train/discharge_target.csv.gz", keep_default_na=False)
#target_valid = pd.read_csv(challenge_data_fp + "valid/discharge_target.csv.gz", keep_default_na=False)
target_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge_target.csv.gz", keep_default_na=False)

#discharge_summ_train = pd.read_csv(challenge_data_fp + "train/discharge.csv.gz", keep_default_na=False)
#discharge_summ_valid = pd.read_csv(challenge_data_fp + "valid/discharge.csv.gz", keep_default_na=False)
discharge_summ_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge.csv.gz", keep_default_na=False)

#radiology_train = pd.read_csv(challenge_data_fp + "train/radiology.csv.gz", keep_default_na=False)
#radiology_valid = pd.read_csv(challenge_data_fp + "valid/radiology.csv.gz", keep_default_na=False)
radiology_test = pd.read_csv(challenge_data_fp + "test_phase_1/radiology.csv.gz", keep_default_na=False)

#edstay_train = pd.read_csv(challenge_data_fp + "train/edstays.csv.gz", keep_default_na=False)
#edstay_valid = pd.read_csv(challenge_data_fp + "valid/edstays.csv.gz", keep_default_na=False)
edstay_test = pd.read_csv(challenge_data_fp + "test_phase_1/edstays.csv.gz", keep_default_na=False)

#edtriage_train = pd.read_csv(challenge_data_fp + "train/triage.csv.gz", keep_default_na=False)
#edtriage_valid = pd.read_csv(challenge_data_fp + "valid/triage.csv.gz", keep_default_na=False)
edtriage_test = pd.read_csv(challenge_data_fp + "test_phase_1/triage.csv.gz", keep_default_na=False)

#eddiags_train = pd.read_csv(challenge_data_fp + "train/diagnosis.csv.gz", keep_default_na=False)
#eddiags_valid = pd.read_csv(challenge_data_fp + "valid/diagnosis.csv.gz", keep_default_na=False)
eddiags_test = pd.read_csv(challenge_data_fp + "test_phase_1/diagnosis.csv.gz", keep_default_na=False)

In [6]:
''' Only populated test set, ignoring training and validating sets
\target_train['split'] = "train"
target_valid['split'] = "valid"
target_test['split'] = "test"
discharge_summ_train['split'] = "train"
discharge_summ_valid['split'] = "valid"
discharge_summ_test['split'] = "test"
radiology_train['split'] = "train"
radiology_valid['split'] = "valid"
radiology_test['split'] = "test"
edstay_train['split'] = "train"
edstay_valid['split'] = "valid"
edstay_test['split'] = "test"
edtriage_train['split'] = "train"
edtriage_valid['split'] = "valid"
edtriage_test['split'] = "test"
eddiags_train['split'] = "train"
eddiags_valid['split'] = "valid"
eddiags_test['split'] = "test"
'''

In [15]:
''' Vimig's code:
target = pd.concat([target_train, target_valid, target_test])
discharge_summ = pd.concat([discharge_summ_train, discharge_summ_valid, discharge_summ_test])
radiology = pd.concat([radiology_train, radiology_valid, radiology_test])
edstay = pd.concat([edstay_train, edstay_valid, edstay_test])
edtriage = pd.concat([edtriage_train, edtriage_valid, edtriage_test])
eddiags = pd.concat([eddiags_train, eddiags_valid, eddiags_test])
'''
# Just reassigning to new variable names so I don't have to rename everything down below, this is only test set now.
target = target_test
discharge_summ = discharge_summ_test
radiology = radiology_test
edstay = edstay_test
edtriage = edtriage_test
eddiags = eddiags_test

In [16]:
target = target.reset_index()
discharge_summ = discharge_summ.reset_index()
radiology = radiology.reset_index()
edstay = edstay.reset_index()
edtriage = edtriage.reset_index()
eddiags = eddiags.reset_index()

In [17]:
discharge_summ

,index,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,0,10000117-DS-22,10000117,27988844,DS,22,2183-09-21 00:00:00,2183-09-29 16:23:00,\nName: ___ Unit No: ___\n...
1,1,10000935-DS-20,10000935,26381316,DS,20,2187-08-27 00:00:00,2187-08-27 16:59:00,\nName: ___ Unit No: ___...
2,2,10000980-DS-22,10000980,24947999,DS,22,2190-11-08 00:00:00,2190-11-09 13:57:00,\nName: ___ Unit No: ___\n \nAdmi...
3,3,10001401-DS-20,10001401,27060146,DS,20,2131-10-05 00:00:00,2131-10-05 16:11:00,\nName: ___ Unit No: ___\n \n...
4,4,10001401-DS-21,10001401,28058085,DS,21,2131-11-15 00:00:00,2131-11-20 19:51:00,\nName: ___ Unit No: ___\n \n...
...,...,...,...,...,...,...,...,...,...
14697,14697,19997293-DS-10,19997293,26366652,DS,10,2123-11-19 00:00:00,2124-07-01 14:15:00,\nName: ___ Unit No: _...
14698,14698,19998330-DS-17,19998330,23151993,DS,17,2178-09-23 00:00:00,2178-09-24 11:43:00,\nName: ___ Unit No: ...
14699,14699,19998497-DS-9,19998497,21557581,DS,9,2145-08-01 00:00:00,2145-08-01 14:13:00,\nName: ___ Unit No: ___\n \...
14700,14700,19998562-DS-21,19998562,26846592,DS,21,2166-04-16 00:00:00,2166-04-17 16:08:00,\nName: ___ Unit No: ___\...


In [18]:
# target['discharge_instructions'] = target['text'].apply(lambda x: re.findall(r'Discharge Instructions:\n(.*?)Followup Instruction', x, re.DOTALL))

discharge_summ['preceding_text'] = discharge_summ['text'].apply(lambda x: re.findall(r'(^.*?)Brief Hospital Course:\s*\n{0,2}(.*?)(?=\n\s*\n{0,2}\s*[A-Z_]+[^\n:]+:\n)', x, re.DOTALL)[0][0])

In [19]:
display(discharge_summ, target)

,index,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,preceding_text
0,0,10000117-DS-22,10000117,27988844,DS,22,2183-09-21 00:00:00,2183-09-29 16:23:00,\nName: ___ Unit No: ___\n...,\nName: ___ Unit No: ___\n...
1,1,10000935-DS-20,10000935,26381316,DS,20,2187-08-27 00:00:00,2187-08-27 16:59:00,\nName: ___ Unit No: ___...,\nName: ___ Unit No: ___...
2,2,10000980-DS-22,10000980,24947999,DS,22,2190-11-08 00:00:00,2190-11-09 13:57:00,\nName: ___ Unit No: ___\n \nAdmi...,\nName: ___ Unit No: ___\n \nAdmi...
3,3,10001401-DS-20,10001401,27060146,DS,20,2131-10-05 00:00:00,2131-10-05 16:11:00,\nName: ___ Unit No: ___\n \n...,\nName: ___ Unit No: ___\n \n...
4,4,10001401-DS-21,10001401,28058085,DS,21,2131-11-15 00:00:00,2131-11-20 19:51:00,\nName: ___ Unit No: ___\n \n...,\nName: ___ Unit No: ___\n \n...
...,...,...,...,...,...,...,...,...,...,...
14697,14697,19997293-DS-10,19997293,26366652,DS,10,2123-11-19 00:00:00,2124-07-01 14:15:00,\nName: ___ Unit No: _...,\nName: ___ Unit No: _...
14698,14698,19998330-DS-17,19998330,23151993,DS,17,2178-09-23 00:00:00,2178-09-24 11:43:00,\nName: ___ Unit No: ...,\nName: ___ Unit No: ...
14699,14699,19998497-DS-9,19998497,21557581,DS,9,2145-08-01 00:00:00,2145-08-01 14:13:00,\nName: ___ Unit No: ___\n \...,\nName: ___ Unit No: ___\n \...
14700,14700,19998562-DS-21,19998562,26846592,DS,21,2166-04-16 00:00:00,2166-04-17 16:08:00,\nName: ___ Unit No: ___\...,\nName: ___ Unit No: ___\...


,index,note_id,hadm_id,discharge_instructions,brief_hospital_course,discharge_instructions_word_count,brief_hospital_course_word_count
0,0,19766998-DS-20,26231944,"Dear Mr. ___,\n\nIt was a pleasure to care for...","___ with PMH HCV, ETOH cirrhosis with ascites,...",145,62
1,1,10336082-DS-11,28542384,"Dear Ms. ___, \n We had the pleasure of taking...",___ yo F with recent lumbar laminectomy c/b MS...,76,466
2,2,10481170-DS-22,26489329,You were admitted to the hospital after a with...,The patient was seen ___ the emergency departm...,396,262
3,3,11576109-DS-14,22641254,"Dear Ms. ___, \n\nIt was a pleasure caring for...","Ms. ___ is a ___ woman with a history of T2DM,...",142,496
4,4,19249697-DS-17,29265750,"Dear Mr. ___, \n\nYou were admitted to ___ wit...",___ with IDDM and h/o provoked PE on apixiban ...,132,396
...,...,...,...,...,...,...,...
14697,14697,18995174-DS-17,27445071,"Dear Mr. ___,\n\nYou were admitted after getti...",___ M PMHx dilated non-ischemic cardiomyopathy...,128,216
14698,14698,13588195-DS-9,26192891,"Dear Mr. ___,\n\n You were admitted to ___ af...","Mr. ___ is a ___ year-old man with CAD, HTN, H...",47,40
14699,14699,10873131-DS-17,26584893,"Dear Ms. ___,\n\nIt was a pleasure taking ___ ...",___ year old female with past medical history ...,260,1014
14700,14700,12332377-DS-11,25623241,"Dear Mr. ___, \n\nYou were admitted to ___ \nb...",Mr. ___ is a ___ male with a past medical hist...,244,462


In [20]:
'''
target_train = target_train.merge(discharge_summ[['note_id', "preceding_text"]], on="note_id", how="left", indicator="indicator")
target_valid = target_valid.merge(discharge_summ[['note_id', "preceding_text"]], on="note_id", how="left", indicator="indicator")
'''
target_test = target_test.merge(discharge_summ[['note_id', "preceding_text"]], on="note_id", how="left", indicator="indicator")


In [21]:
#target_train

In [23]:
# Milgram is not letting me import tiktoken??
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [24]:
#target_train['tokens'] = target_train.apply(lambda row: num_tokens_from_string(row['brief_hospital_course'] + row['preceding_text'], "cl100k_base"), axis=1)

In [25]:
#target_train['tokens'].describe()

In [26]:
# we do this in the mistrial finetune notebook so don't have to do it here 
# target_train['prompt_with_preceding_text'] = "Summarize the following patient hospital encounter into a brief hospital course:\n\n" + target_train['preceding_text']
# target_valid['prompt_with_preceding_text'] = "Summarize the following patient hospital encounter into a brief hospital course:\n\n" + target_valid['preceding_text']

target_test['prompt_with_preceding_text'] = "Summarize the following patient hospital encounter into a brief hospital course:\n\n" + target_test['preceding_text']


In [27]:
target_test.reset_index(drop = True, inplace = True)
target_test.head()

,note_id,hadm_id,discharge_instructions,brief_hospital_course,discharge_instructions_word_count,brief_hospital_course_word_count,preceding_text,indicator,prompt_with_preceding_text
0,19766998-DS-20,26231944,"Dear Mr. ___,\n\nIt was a pleasure to care for...","___ with PMH HCV, ETOH cirrhosis with ascites,...",145,62,\nName: ___ Unit No: ___\n ...,both,Summarize the following patient hospital encou...
1,10336082-DS-11,28542384,"Dear Ms. ___, \n We had the pleasure of taking...",___ yo F with recent lumbar laminectomy c/b MS...,76,466,\nName: ___ Unit No: ___\n \nA...,both,Summarize the following patient hospital encou...
2,10481170-DS-22,26489329,You were admitted to the hospital after a with...,The patient was seen ___ the emergency departm...,396,262,\nName: ___ Unit No: ___\n ...,both,Summarize the following patient hospital encou...
3,11576109-DS-14,22641254,"Dear Ms. ___, \n\nIt was a pleasure caring for...","Ms. ___ is a ___ woman with a history of T2DM,...",142,496,\nName: ___ Unit No: ___\n ...,both,Summarize the following patient hospital encou...
4,19249697-DS-17,29265750,"Dear Mr. ___, \n\nYou were admitted to ___ wit...",___ with IDDM and h/o provoked PE on apixiban ...,132,396,\nName: ___ Unit No: ...,both,Summarize the following patient hospital encou...


In [28]:
#target_train.rename({"preceding_text":"input", "brief_hospital_course":"output"}, axis=1)[['input', "output"]].to_json(challenge_data_fp + "train/simple_train.json", orient="records")

In [29]:
#target_valid.rename({"preceding_text":"input", "brief_hospital_course":"output"}, axis=1)[['input', "output"]].to_json(challenge_data_fp + "valid/simple_valid.json", orient="records")

In [32]:
target_test.rename({"preceding_text":"input", "brief_hospital_course":"output"}, axis=1)[["note_id", "hadm_id", 'input', "output"]].to_json("simple_test.json", orient="records")